# Regularization (正则化)

#### 问题：什么是过拟合？

在训练数据不够多时，或者 overtraining 时，常常会导致 overfitting （过拟合）。其直观的表现如下图所示，随着训练过程的进行，模型复杂度增加，在 training data 上的 error 逐渐减小，但是在验证集上的 error 却反而渐渐增大 —— 因为训练出来的网络过拟合了训练集，对训练集外的数据却降低了有效性。

![Overfitting](imgs/overfitting.jpg)

#### 问题：为什么要将原始数据集拆分为三部分 training data、validation data 和 testing data ？

本质上，这个问题是回答为什么需要 validation data 。

validation data 的作用就是为了避免过拟合，在训练过程中，通常用它来确定一些超参数，比如根据 validation data 上的 accuracy 来确定 early stopping 的 epoch 大小、根据 validation data 确定 learning rate 等等。

#### 问题：为什么不直接在 testing data 上确定超参数呢？

因为如果在 testing data 做这些，那么随着训练的进行，网络实际上就是在逐渐地过拟合 testing data ，导致最后得到的 testing accuracy 没有任何参考意义。因此， training data 的作用是计算梯度更新权重，validation data 用来确定超参数和避免过拟合，testing data 则给出一个 accuracy 以判断网络的好坏。

#### 问题：有哪些常见的正则化方法？

L1 和 L2 正则化，Early Stopping，数据集扩增 (Data Augmentation)，Dropout 等。

#### 问题：什么是正则化？

简单的说，就是在某种条件约束下求解损失函数的最值。表现形式为加到损失函数后面的额外项（也叫惩罚项），通常有 $\iota_1$-norm (范数) 和 $\iota_2$-norm 两种。

### L1 and L2 Regularization

#### 问题：什么是 L1 正则化？什么是 L2 正则化？

* L1 正则化是指权重向量 $w$ 中各个元素的绝对值之和，通常表示为 $||w||_1$
* L2 正则化是指权重向量 $w$ 中各个元素的平方和然后再求平方根，通常表示为 $||w||_2$

一般会在正则化项之前加一个系数，这个系数需要用户指定。

#### 问题：什么是稀疏矩阵？稀疏矩阵有什么作用？为什么稀疏矩阵可以用来做特征选择？

稀疏矩阵指的是很多元素为 0 ，只有少数元素是非零值的矩阵，即得到的线性回归模型的大部分系数都是 0 。

稀疏矩阵的作用：

* 模型压缩：特别是针对于移动和嵌入式设备；
* 特征选择：只需要关注系数是非零项的特征。

通常机器学习中特征数量很多，稀疏模型表示绝大部分特征对预测值没有贡献，或者贡献很小（权重接近于 0 ），即使去掉对模型也没什么影响，这样就只需要关注系数是非零项的特征，这就是稀疏矩阵与特征选择之间的关系。

#### 问题：为什么 L1 正则化可以产生稀疏矩阵？为什么 L2 正则化不具有稀疏性？

假设有如下带L1正则化的损失函数：

$$J=J_0+\alpha\sum_w|w|$$

其中 $J_0$ 是原始的损失函数，加号后面的一项是 L1 正则化项，$\alpha$ 是正则化系数。注意到 L1 正则化是权值的绝对值之和，$J$ 是带有绝对值符号的函数，因此 $J$ 是不完全可微的。机器学习的任务就是要通过优化方法求出损失函数的最小值。当我们在原始损失函数 $J_0$ 后添加 L1 正则化项时，相当于对 $J_0$ 做了一个约束。令 $L=\alpha\sum_w|w|$ ，则 $J=J_0+L$，此时我们的任务变成在 $L$ 约束下求出 $J_0$ 取最小值的解。考虑二维的情况，即只有两个权值 $w_1$ 和 $w_2$ ，此时 $L=|w_1|+|w_2|$ 对于梯度下降法，求解 $J_0$ 的过程可以画出等值线，同时 L1 正则化的函数 L 也可以在 $w_1w_2$ 的二维平面上画出来。如下图：

![L1 Regularization](imgs/l1_regularization.png)

图中等值线是 $J_0$ 的等值线，黑色方形是 $L$ 函数的图形。在图中，当 $J_0$ 等值线与 $L$ 图形首次相交的地方就是最优解。上图中 $J_0$ 与 $L$ 在 $L$ 的一个顶点处相交，这个顶点就是最优解。注意到这个顶点的值是 $(w_1,w_2)=(0,w)$。可以直观想象，因为 $L$ 函数有很多『突出的角』（二维情况下四个，多维情况下更多），$J_0$ 与这些角接触的机率会远大于与 $L$ 其它部位接触的机率，而在这些角上，会有很多权值等于 0 ，这就是为什么 L1 正则化可以产生稀疏模型，进而可以用于特征选择。

而正则化前面的系数 $\alpha$，可以控制 $L$ 图形的大小。$\alpha$ 越小，$L$ 的图形越大（上图中的黑色方框）；$\alpha$ 越大，$L$ 的图形就越小，可以小到黑色方框只超出原点范围一点点，这是最优点的值 $(w_1,w_2)=(0,w)$ 中的 $w$ 可以取到很小的值。

类似，假设有如下带 L2 正则化的损失函数：

$$J=J_0+\alpha\sum_ww^2$$

同样可以画出在二维平面上的图形，如下：

![L2 Regularization](imgs/l2_regularization.png)

二维平面下 L2 正则化的函数图形是个圆，与方形相比，被磨去了棱角。因此 $J_0$ 与 $L$ 相交时使得 $w_1$ 或 $w_2$ 等于零的机率小了许多，这就是为什么 L2 正则化不具有稀疏性的原因。

#### 问题：为什么过拟合倾向于使得权重尽可能小？

参数值小的模型比较简单，能适应不同的数据集，也在一定程度上避免了过拟合现象。对于一个线性回归方程，若参数很大，那么只要数据偏移一点点，就会对结果造成很大的影响；但如果参数足够小，数据偏移得多一点也不会对结果造成什么影响。

#### 问题：为什么 L2 正则化可以获得值很小的参数？

注：L2 正则化因此也叫做权重衰减（Weight Decay）。

以线性回归中的梯度下降法为例。假设要求的参数为 $\theta$ ，$h_\theta(x)$ 是假设函数，那么线性回归的代价函数如下：

$$J(\theta)=\frac{1}{2m}\sum_{i=1}^{m}(h_\theta(x^{(i)})−y^{(i)})$$

那么在梯度下降法中，最终用于迭代计算参数 $\theta$ 的迭代式为：

$$\theta_j:=\theta_j−\alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})−y^{(i)})x_{j}^{(i)}$$

其中 $\alpha$ 是 learning rate 。 上式是没有添加 L2 正则化项的迭代公式，如果在原始代价函数之后添加 L2 正则化，则迭代公式会变成下面的样子：

$$\theta_j:=\theta_j(1−\alpha\frac{\lambda}{m})−\alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})−y^{(i)})x_j^{(i)}$$

其中 $\lambda$ 就是正则化参数。从上式可以看到，与未添加 L2 正则化的迭代公式相比，每一次迭代，$\theta_j$ 都要先乘以一个小于 1 的因子，从而使得 $\theta_j$ 不断减小，因此总得来看，$\theta$ 是不断减小的。但是考虑到后面的导数项，$w$ 最终的值可能增大也可能减小。

L1 正则化一定程度上也可以防止过拟合。因为当 L1 的正则化系数很小时，得到的最优解会很小，可以达到和 L2 正则化类似的效果。

#### L2 正则化推导过程

$$C=C_0+\frac{\lambda}{2n}\sum_{w}w^{2}$$

求导，

$$\frac{\partial C}{\partial w}=\frac{\partial C_0}{\partial w}+\frac{\lambda}{n}w$$

$$\frac{\partial C}{\partial b}=\frac{\partial C_0}{\partial b}$$

即，L2 正则化对 $b$ 的更新没有影响，但是对于 $w$ 的更新有影响。

$$w\to w-\eta\frac{\partial C_0}{\partial w}-\frac{\eta\lambda}{n}w$$

$$=(1-\frac{\eta\lambda}{n})w-\eta\frac{\partial C_0}{\partial w}$$

对于基于 mini-batch 的随机梯度下降，更新的公式稍有不同：

$$w\to (1-\frac{\eta\lambda}{n})w-\frac{\eta}{m}\sum_x\frac{\partial C_x}{\partial w}$$

$$b\to b-\frac{\eta}{m}\sum_x\frac{\partial C_x}{\partial w}$$

即，后面的一项变成所有导数的加和，乘以 $\eta$ 再除以 $m$ ，$m$ 是一个 mini-batch 中样本的个数。

#### L1 正则化推导过程

$$C=C_0+\frac{\lambda}{n}\sum_{w}|w|$$

求导，

$$\frac{\partial C}{\partial w}=\frac{\partial C_0}{\partial w}+\frac{\lambda}{n}sgn(w)$$

上式中，$sgn(w)$ 表示 $w$ 的符号，权重 $w$ 的更新规则为：

$$w\to w-\frac{\eta\lambda}{n}sgn(w)-\eta\frac{\partial C_0}{\partial w}$$

即，比原始的更新规则多出了 $\frac{\eta\lambda}{n}sgn(w)$ 这一项。当 $w$ 为正时，更新后的 $w$ 变小；当 $w$ 为负时，更新后的 $w$ 变大。因此它的效果就是让 $w$ 靠近 0 ，也就是减小了网络复杂度，防止过拟合。

##### 问题：当 $w$ 为 0 的时候怎么办？

当 $w$ 等于 0 时，$|w|$ 是不可导的，只能按照原始的未经正则化的方法去更新 $w$ ，这就相当于去掉 $\frac{\eta\lambda}{n}sgn(w)$ 这一项，所以可以规定 $sgn(0)=0$ ，这样就把 $w=0$ 的情况也统一进来了。（在编程的时候，令$sgn(0)=0,sgn(w>0)=1,sgn(w<0)=-1$）

#### 问题：如何选择 L1 的正则化参数？

通常越大的 $\lambda$ 可以让代价函数在参数为 0 时取到最小值 <sup>[1]</sup>。假设有如下带 L1 正则化项的代价函数：

$$F(x)=f(x)+\lambda||x||_1$$

其中 $x$ 是要估计的参数，相当于上文中提到的 $w$ 以及 $\theta$ 。 注意 L1 正则化在某些位置是不可导的，当 $\lambda$ 足够大时可以使得 $F(x)$ 在 $x=0$ 时取到最小值。如下图：

![L1 Regularization Parameter](imgs/l1_regularization_parameter.png)

#### 问题：如何选择 L2 的正则化参数？

$$\theta_j:=\theta_j(1−\alpha\frac{\lambda}{m})−\alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})−y^{(i)})x_j^{(i)}$$

从公式中看出，$\lambda$ 越大，$\theta$ 衰减地越快。

从 L2 正则化的可视化图中看出，$\lambda$ 越大，L2 圆的半径越小，最后求得代价函数最值时各参数也会变得很小。

### Data Augmentation

### Dropout

## References

1. https://www.quora.com/What-is-the-difference-between-L1-and-L2-regularization-How-does-it-solve-the-problem-of-overfitting-Which-regularizer-to-use-and-when/answer/Kenneth-Tran